In [3]:
"""Sentiment Analysis of a sentence using Sentiwordnet scores."""
# Extended the code available at https://stackoverflow.com/questions/38263039/sentiwordnet-scoring-with-python
from sys import argv
from nltk import word_tokenize
from nltk import pos_tag
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer


def get_wordnet_pos(word):
    """Convert the PENN Treebank POS tags to Wordnet tags."""
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wn.ADJ,
                "N": wn.NOUN,
                "V": wn.VERB,
                "R": wn.ADV}
    return tag_dict.get(tag, wn.NOUN)


def get_sentiment_score_of_review(sentence, common=True):
    # 1. Tokenize the sentence
    # 2. Lemmatize the tokens
    # 3. Get the POS tag for each token
    # 4. Get the synsets for each token
    # 5. Get the SentiWordNet score for each synset
    # 6. Calculate the sentiment score for the sentence
    # 7. Return the sentiment score
    tokens = word_tokenize(sentence)    

    lemmatizer = WordNetLemmatizer()

    sentiment_score = 0.0
    for word in tokens:
        tag = get_wordnet_pos(word)
        item_res = lemmatizer.lemmatize(word, tag)
        if not item_res:
            continue
        
        synsets = wn.synsets(item_res, pos=tag)
        if len(synsets) == 0:
            print("No synsets exist for the", word)
            continue
        
        # Take the first, the most common
        if common:
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            sentiment_score += swn_synset.pos_score() - swn_synset.neg_score()
        else:
            # Else take the best sentiment score
            sentiment_scores = []
            for synset in synsets:
                swn_synset = swn.senti_synset(synset.name())
                senti_score = swn_synset.pos_score() - swn_synset.neg_score()
                sentiment_scores.append(senti_score)
            sentiment_score += max(sentiment_scores)

    return sentiment_score


In [4]:
sentence = "The movie was not that good, but the acting was great."
sentiment_score = get_sentiment_score_of_review(sentence, common=False)
print('The input sentence is:', sentence)
print('Sentiment score of the sentence is:', sentiment_score)
if sentiment_score > 0:
    print('The sentiment of the sentence is positive')
elif sentiment_score == 0:
    print('The sentiment of the sentence is neutral')
else:
    print('The sentiment of the sentence is negative')

No synsets exist for the The
No synsets exist for the that
No synsets exist for the ,
No synsets exist for the but
No synsets exist for the the
No synsets exist for the .
The input sentence is: The movie was not that good, but the acting was great.
Sentiment score of the sentence is: 1.875
The sentiment of the sentence is positive


In [5]:
! pip install vaderSentiment
# Importing the SentimentIntensityAnalyzer from the vaderSentiment package
# This package is used for sentiment analysis based on the VADER (Valence Aware Dictionary
# and sEntiment Reasoner) algorithm.
# VADER is particularly effective for analyzing sentiments expressed in social media,
# such as tweets, reviews, and other short texts.
# It provides a simple and efficient way to determine the sentiment polarity of a given text.
# It assigns a sentiment score to the text, indicating whether the sentiment is positive,
# negative, or neutral.
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer    

In [6]:
def sentiment_scores(sentence):
    """Get the sentiment scores of a sentence using VADER."""
    # Create a SentimentIntensityAnalyzer object
    # This object will be used to calculate the sentiment scores
    # for the input sentence.
    # It uses the VADER algorithm to analyze the sentiment of the text.
    # The VADER algorithm is designed to work well with short texts,
    # such as tweets, reviews, and other social media content.
    # It provides a compound score that summarizes the overall sentiment,
    # as well as individual scores for positive, negative, and neutral sentiments.
    # The compound score is a normalized score that ranges from -1 (most negative) to 1 (most positive).
    # The positive, negative, and neutral scores represent the proportion of the text that is positive, negative, and neutral, respectively.
    # The scores are calculated based on the presence of specific words and their associated sentiment values.
    # The SentimentIntensityAnalyzer object is initialized with a pre-defined lexicon of words and their sentiment values.
    # It uses this lexicon to analyze the sentiment of the input text.
    # The object can be reused for multiple sentences without needing to reinitialize it.
    # This makes it efficient for processing large amounts of text.
    # The SentimentIntensityAnalyzer object can be used to analyze the sentiment of any text
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(sentence)

    print(f"Sentiment Scores: {sentiment_dict}")
    print(f"Negative Sentiment: {sentiment_dict['neg']*100}%")
    print(f"Neutral Sentiment: {sentiment_dict['neu']*100}%")
    print(f"Positive Sentiment: {sentiment_dict['pos']*100}%")
    
    if sentiment_dict['compound'] >= 0.05:
        print("Overall Sentiment: Positive")
    elif sentiment_dict['compound'] <= -0.05:
        print("Overall Sentiment: Negative")
    else:
        print("Overall Sentiment: Neutral")

In [7]:
print("\nStatement:")
sentence = "Shweta played well in the match as usual."
sentiment_scores(sentence)

print("\nStatement:")
sentence = "I am feeling sad today."
sentiment_scores(sentence)


Statement:
Sentiment Scores: {'neg': 0.0, 'neu': 0.571, 'pos': 0.429, 'compound': 0.5423}
Negative Sentiment: 0.0%
Neutral Sentiment: 57.099999999999994%
Positive Sentiment: 42.9%
Overall Sentiment: Positive

Statement:
Sentiment Scores: {'neg': 0.408, 'neu': 0.395, 'pos': 0.197, 'compound': -0.3818}
Negative Sentiment: 40.8%
Neutral Sentiment: 39.5%
Positive Sentiment: 19.7%
Overall Sentiment: Negative
